In [38]:
from cartopy.io.shapereader import Reader
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, WidgetBox, ColorMapper
from bokeh.io import output_notebook, push_notebook, show
from bokeh.plotting import figure, Figure
from bokeh.layouts import column, widgetbox
from bokeh.palettes import Viridis256 as palette
# from bokeh.palettes import Magma256 as palette
from bokeh.models.widgets import Panel, Tabs, Slider, RadioButtonGroup
import os
import pandas as pd
import pickle
import numpy as np
from ipywidgets import interact, SelectionSlider, Select, ToggleButtons

output_notebook()

Loading BokehJS ...

In [29]:
pickle_data = open(os.path.join(os.getcwd(), 'data', 'processed', 'clean_data.pickle'), 'rb')
data = pickle.load(pickle_data)

geoids = list(set(data.index))

In [30]:
# read the shapefile
tracts = Reader(r'census_tract_shapefile/austin_metro_tracts.shp')
# extract the x's and y's (bokeh preferred format is list of lists)
xs, ys, names, tract = [], [], [], []
for r, g in zip(tracts.records(), tracts.geometries()):
    if int(r.attributes['GEOID']) in geoids:
        xs.append(list(g[0].exterior.coords.xy[0]))
        ys.append(list(g[0].exterior.coords.xy[1]))
        names.append(int(r.attributes['GEOID']))
        tract.append(float(r.attributes['GEOID'][-6:]) / 100)
print(len(xs), len(ys), len(names), len(tract))

343 343 343 343


In [31]:
# set up the color mapper
color_mapper = LogColorMapper(palette=palette)
# choose which bokeh tools are included
TOOLS = 'pan, wheel_zoom, hover, reset, save'

In [32]:
def make_source(year):
    source = ColumnDataSource(data = dict(
        x=xs,
        y=ys,
        name=names,
        tract=tract,
        income=data[data['year'] == year].loc[names, 'income_index'].tolist(),
    ))
    
    return source

def update_data(year):
    updata = dict(
        x=xs,
        y=ys,
        name=names,
        tract=tract,
        income=data[data['year'] == year].loc[names, 'income_index'].tolist(),
    )
    
    return updata

In [33]:
source = make_source('2000')

p_income = figure(title='Income', tools=TOOLS, x_axis_location=None,
                  y_axis_location=None)
r_income = p_income.patches('x', 'y', source=source, fill_color={'field': 'income', 'transform': color_mapper},
                 fill_alpha=0.7, line_color='black', line_width=0.5)
hover_income = p_income.select_one(HoverTool)
hover_income.point_policy = 'follow_mouse'
hover_income.tooltips = [
    ('GeoID', '@name'),
    ('Census Tract', '@tract'),
    ('Lon, Lat', '$x, $y'),
    ('Income Index', '@income{0,0}')
]
tab_income = Panel(child=p_income, title='Income')

In [108]:
def print_value(attr, old, new):
    attr = new

radio_group = RadioButtonGroup(labels=['2000', '2009', '2011', '2012', '2013',
                                       '2014', '2015', '2016'], active=0)

radio_group.on_change('active', lambda attr, old, new: update()) ### soooo close!!!

In [106]:
tabs = Tabs(tabs=[tab_income])

t = show(widgetbox([radio_group, tabs], sizing_mode='scale_both'), notebook_handle=True)

In [81]:
r_income.data_source.data = update_data('2016')
push_notebook(handle=t)

In [47]:
widgetbox?

In [107]:
radio_group.active

0